In [2]:
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import numpy as np
from time import sleep, time
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from evidently.report import Report
from evidently import ColumnMapping
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric
import mlflow
from mlflow import MlflowClient
from mlflow.entities import ViewType
import mlflow.sklearn
from pprint import pprint
from prefect import flow, task

In [3]:
AVAILABLE_DATA_PATH = './data/available'
MODEL_NAME = 'car-price-predictor'
DEBUG = True
if DEBUG:
    !rm ./data/available/*

In [4]:
# Connect to the MLFlow tracking server
mlflow.set_tracking_uri("http://0.0.0.0:5000")

In [12]:
@task()
def load_existing_dataset(input_folder):
    assert os.path.exists(input_folder), f"Input location {input_folder} does not exist."
    files_list = glob.glob(os.path.join(input_folder, "*.csv"))
    print(f"Found {len(files_list)} data files.")
    if len(files_list) == 0:
        print("No data available.")
        return None
    df = pd.concat(map(pd.read_csv, files_list))
    print("Loaded available data.")
    return df

@task()
def train_model(df, feature_columns, target_column, model_type = 'LinearRegressor'):

    df=df.dropna().reset_index(drop=True)
    X = df[feature_columns].values
    #print(np.shape(X))
    y = df[target_column].values
    #print(np.shape(y))
    
    with mlflow.start_run():
        mlflow.log_param("model_type", model_type)
        mlflow.log_param("latest_data_year", df['year'].max())
        mlflow.log_param("feature_columns", feature_columns)
        mlflow.log_param("target_column", target_column)

        if model_type == "LinearRegressor":
            model = LinearRegression().fit(X, y)
        elif model_type == "GradientBoostingRegressor":
            model = GradientBoostingRegressor().fit(X, y)
        elif model_type == "RandomForestRegressor":
            model = RandomForestRegressor().fit(X, y)

        y_pred = model.predict(X)
        rmse = mean_squared_error(y, y_pred, squared=False)
        score = model.score(X,y)

        mlflow.log_metric("train_rmse", rmse)
        mlflow.log_metric("train_score", score)
        mlflow.sklearn.log_model(model, "model")

    
    #
    #model = GradientBoostingRegressor().fit(X, y)

    #print("Model score on training data:", model.score(X, y))
    return model
    #print(model.coef_)
    #print(model.intercept_)
    '''
    print(model.predict(df[df['year']<85][feature_cols].values))
    plt.plot(model.predict(df[df['year']<85][feature_cols].values),label='Predicted', alpha=0.5)
    plt.plot(df[df['year']<85][target_col].values,label='Measured', alpha=0.5)
    plt.legend()
    plt.show()
    '''


@task()
def evaluate_model(model, df_test, feature_columns, target_column, display=True):
    df_test=df_test.dropna().reset_index(drop=True)
    X = df_test[feature_columns].values
    y = df_test[target_column].values
    print("Model score :", model.score(X, y))
    y_pred = model.predict(X)
    rmse = mean_squared_error(y, y_pred, squared=False)
    print("Model rmse: ", rmse)
    if display:
        plt.plot(model.predict(df_test[feature_columns].values),label='Predicted', alpha=0.5)
        plt.plot(df_test[target_column].values,label='Measured', alpha=0.5)
        plt.legend()
        plt.show()

@task()
def compare_datasets(df, latest_training_data, feature_columns, target_column, display=True):
    #print(df[feature_columns + [target_column]].mean())
    #print(df_new[feature_columns + [target_column]].mean())
    reference_data = df[df['year'] <= latest_training_data]
    current_data = df[df['year'] > latest_training_data]
    
    reference_data_mean_values = np.array(reference_data[feature_columns + [target_column]].mean().values)
    current_data_new_mean_values = np.array(current_data[feature_columns + [target_column]].mean().values)
    if display:
        plt.plot(current_data_new_mean_values / reference_data_mean_values * 100, label='new/available data [%]')
        plt.xticks(list(range(len(reference_data_mean_values))), 
                   reference_data[feature_columns + [target_column]].mean().index, 
                   rotation=45)
        plt.legend()
        plt.show()

@task()
def calculate_monitoring_metrics(model, df, latest_training_data, 
                                 numerical_features = None, 
                                 categorical_features = None, 
                                 target_column = None):
    print(latest_training_data)
    print(df['year'].min(), df['year'].max())
    all_features_list = []
    if numerical_features:
        all_features_list = all_features_list + numerical_features
    if categorical_features:
        all_features = all_features_list + categorical_features
        
    column_mapping = ColumnMapping(
        prediction='prediction',
        numerical_features=numerical_features,
        categorical_features=categorical_features,
        target=target_column
    )
    report = Report(metrics = [
        ColumnDriftMetric(column_name='prediction'),
        DatasetDriftMetric(),
        DatasetMissingValuesMetric()
    ])
    
    reference_data = df[df['year'] <= latest_training_data]
    current_data = df[df['year'] > latest_training_data]
    current_data['prediction'] = model.predict(current_data[all_features_list].fillna(0))
    reference_data['prediction'] = model.predict(reference_data[all_features_list].fillna(0))

    report.run(reference_data = reference_data, current_data = current_data, column_mapping=column_mapping)

    result = report.as_dict()

    prediction_drift = result['metrics'][0]['result']['drift_score']
    num_drifted_columns = result['metrics'][1]['result']['number_of_drifted_columns']
    share_missing_values = result['metrics'][2]['result']['current']['share_of_missing_values']
    
    print("prediction_drift:", prediction_drift)
    print("num_drifted_columns:", num_drifted_columns)
    print("share_missing_values:", share_missing_values)


@flow()
def main_training_flow(available_data_path, model_name, display=True):
    feature_cols = ['horsepower','places', 'doors','speed', 'consumption', 'acceleration']
    #feature_cols = ['horsepower']
    target_col = 'price_eur'
    latest_training_data = 1900
    latest_available_data = 1900
    model = None
    client = MlflowClient()
    if DEBUG: 
        # Delete a registered model along with all its versions
        try:
            client.delete_registered_model(name=model_name)
        except:
            print(f"Model {MODEL_NAME} not present in the registry.")

    model = None
    print("----- Iteration -----")

    # Load data
    df = load_existing_dataset(available_data_path)
    if df is None:
        print("No available data for training.")
        return
    latest_available_data = df['year'].max()
        
    if df is not None and latest_training_data < df['year'].max() and latest_training_data > df['year'].min():
        #print(latest_training_data, df['year'].max() , df['year'].min())
        compare_datasets(df, latest_training_data, feature_cols, target_col, display=display)
        
    # Monitoring
    if model is not None and df is not None:
        calculate_monitoring_metrics(model, df.copy(), latest_training_data, numerical_features = feature_cols)

    # Get the latest model version from the registry
    for rm in client.search_registered_models():
        if rm.latest_versions[0].run_id.name != model_name:
            continue
        run_id = rm.latest_versions[0].run_id
        run_obj = mlflow.get_run(run_id)
        run_metrics = run_obj.data.metrics
        run_params = run_obj.data.params
        latest_training_data = int(run_params['latest_data_year'])
        print(run_params)
        
        # Load the model
        model_uri = "runs:/{}/model".format(run_id)
        try:
            model = mlflow.sklearn.load_model(model_uri)
        except:
            print("No model found.")
    '''
    model_version = "latest"
    try:
        model = mlflow.sklearn.load_model(model_uri=f"models:/{model_name}/{model_version}")
    except:
        print(f"Model {model_name}:{model_version} was not found.")
    '''
    #if df is not None:
    #    model = train_model(df, feature_columns=feature_cols, target_column=target_col)
    #    latest_training_data = df['year'].max()
        
    if model is not None:
        evaluate_model(model, df, feature_columns=feature_cols, target_column=target_col, display=display)
        
    # (Re)train model
    EXPERIMENT_NAME = f"car-price-{int(time())}"
    mlflow.set_experiment(EXPERIMENT_NAME)
    for model_type in ["LinearRegressor", "GradientBoostingRegressor", "RandomForestRegressor"]:
        model = train_model(df, feature_columns=feature_cols, target_column=target_col, model_type = model_type)
        evaluate_model(model, df, feature_columns=feature_cols, target_column=target_col, display=display)

    # Select the model with the lowest test RMSE
    experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
    best_run = client.search_runs(
            experiment_ids=experiment.experiment_id,
            filter_string="",
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=1,
            order_by=["metrics.test_rmse ASC"],
        )[0]
    #print(best_run)
    
    # Register the best model
    
    result = mlflow.register_model(
        f"runs:/{best_run.info.run_id}/model", model_name
    )

/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: UserWarning: A task named 'load_existing_dataset' and defined at '/tmp/ipykernel_3489/3025492780.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: UserWarning: A task named 'train_model' and defined at '/tmp/ipykernel_3489/3025492780.py:13' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: UserWarning: A task named 'evaluate_model' and defined at '/tmp/ipykernel_3489/3025492780.py:60' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/usr/local/lib/python3.8/site-packages/prefect/tasks.py:326: 

In [14]:
main_training_flow(available_data_path = AVAILABLE_DATA_PATH, model_name=MODEL_NAME, display=False)

17:55:41.607 | INFO    | prefect.engine - Created flow run 'gray-pegasus' for flow 'main-training-flow'

Model car-price-predictor not present in the registry.
----- Iteration -----


17:55:42.090 | INFO    | Flow run 'gray-pegasus' - Created task run 'load_existing_dataset-0' for task 'load_existing_dataset'

17:55:42.102 | INFO    | Flow run 'gray-pegasus' - Executing 'load_existing_dataset-0' immediately...

Found 1 data files.
Loaded available data.


...

Model score : 0.9442498085618035
Model rmse:  218.00005808750328


17:56:21.510 | INFO    | Task run 'evaluate_model-2' - Finished in state Completed()

Successfully registered model 'car-price-predictor'.
2023/08/02 17:56:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: car-price-predictor, version 1
Created version '1' of model 'car-price-predictor'.


17:56:22.265 | INFO    | Flow run 'gray-pegasus' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `LinearRegression`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `GradientBoostingRegressor`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='res

In [ ]:
# Test service 